# Cross-Validation and the Bootstrap
# STAT318/462 Lab4
### Cross-Validation and the Bootstrap
In this lab you will work through Section 5.3 of the course textbook, An Introduction to Statistical Learning (there is a link to this textbook on the Learn page). 

# The Validation Set Approach
The first thing to do is to load the necessary package and briefly explore the data set called **Auto**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [ ]:
auto = pd.read_csv("Auto.csv", na_values="?") 
auto = auto.dropna()

np.random.seed(10)
auto.shape

In [ ]:
auto.columns

In [ ]:
auto[0:4]

Spliting the datat into training and test using sklearns train_test_split

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(auto, test_size=0.5)

Fit first order polynomial

In [ ]:
from statsmodels.formula.api import ols
lr = ols(formula="mpg ~ horsepower", data=train).fit()
np.mean((test["mpg"] - lr.predict(test)) ** 2)

Fit a second order polynomial

In [ ]:
def poly(x, p):
    x = np.array(x)
    x = np.transpose(np.vstack((x ** k for k in range(p + 1))))
    x = np.linalg.qr(x)[0][:, 1:]
    return x

lr2 = ols(formula="mpg ~ poly(horsepower, 2)", data=train).fit()
np.mean((test["mpg"] - lr2.predict(test)) ** 2)

Fit a third order polynomial

In [ ]:
lr3 = ols(formula="mpg ~ poly(horsepower, 3)", data=train).fit()
np.mean((test["mpg"] - lr3.predict(test)) ** 2)

In [ ]:
test = test.sort_values(by="horsepower")


ax = auto.plot.scatter("horsepower", "mpg", s=40, c="black")
ax.scatter(train["horsepower"], train["mpg"], c="red")
ax.plot(test["horsepower"], lr.predict(test), lw=3)
ax.plot(test["horsepower"], lr2.predict(test), c="green")
ax.plot(test["horsepower"], lr3.predict(test), c="yellow")

### k-Fold Cross-Validation
Here we are going to do the k-Fold Cross-Validation

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
import sklearn
np.random.seed(1)


cv_error = np.zeros(10)

y = np.asarray(auto["mpg"]).reshape(-1, 1)

for i in range(1, 11):
    x = np.asarray(poly(auto["horsepower"], i))

    scores = cross_val_score(LinearRegression(), x, y, cv=10, scoring="neg_mean_squared_error")
    cv_error[i - 1] =-np.mean(scores)
cv_error

In [ ]:
plt.plot(range(1, 11), cv_error)
plt.xlabel("Order")
plt.ylabel("Error")

### The Bootstrap

Let’s bootstrap some “data set”.

A function to calculate the optimal value (minimum variance)

In [ ]:
portfolio = pd.read_csv("Portfolio.csv") 

In [ ]:
def alpha(x, y):
    return ((np.var(y) - np.cov(x, y)) / (np.var(x) + np.var(y) - 2 * np.cov(x, y)))[0, 1]

check the function

In [ ]:
alpha(portfolio["X"], portfolio["Y"])

In [ ]:
portfolio_100 = portfolio[:100]
alpha(portfolio_100["X"], portfolio_100["Y"])

one iteration

In [ ]:
np.random.seed(1)
portfolio_sample = portfolio.sample(100, replace=True)
alpha(portfolio_sample["X"], portfolio_sample["Y"])

the bootstrap

In [ ]:
portfolio_sample = portfolio.sample(100, replace=True)
samples = [portfolio.sample(len(portfolio), replace=True) for _ in range(1000)]
alphas = [alpha(p["X"], p["Y"]) for p in samples]

pd.DataFrame({"a": alphas}).describe()

In [ ]:
plt.hist(alphas, bins=20, density=True)
plt.xlabel("alpha")
plt.ylabel("Density")
plt.title("Histogram of alpha")

### Estimating the accuracy of a linear regression model

For the Auto data set, we want to use bootstrap to estimate the accuracy of the linear regression model

In [ ]:
def boot_func(data):
    lr = ols(formula="mpg ~ horsepower", data=data).fit()
    return lr.params

In [ ]:
boot_func(auto[:392])

In [ ]:
boot_func(auto.sample(392, replace=True))

Perform bootstrapping, and summarize the result

In [ ]:
samples = [auto.sample(len(auto), replace=True) for _ in range(100)]
intercepts, slopes = zip(*[boot_func(s) for s in samples])
pd.DataFrame({"intercept": intercepts, "slope": slopes}).describe()